[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/fusion_retrieval.ipynb)

# 文档搜索中的融合检索

## 概述

此代码实现了一个融合检索系统，将基于向量的相似性搜索与基于关键词的BM25检索相结合。该方法旨在利用两种方法的优势来提高文档检索的整体质量和相关性。

## 动机

传统的检索方法通常依赖于语义理解（基于向量）或关键词匹配（BM25）。每种方法都有其优势和劣势。融合检索旨在结合这些方法，创建一个更强大和准确的检索系统，能够有效处理更广泛的查询。

## 关键组件

1. PDF处理和文本分块
2. 使用FAISS和OpenAI嵌入创建向量存储
3. 为基于关键词的检索创建BM25索引
4. 结合两种方法的自定义融合检索函数

## 方法详情

### 文档预处理

1. 使用RecursiveCharacterTextSplitter加载PDF并将其分割成块。
2. 通过将't'替换为空格来清理块（可能解决特定的格式问题）。

### 向量存储创建

1. 使用OpenAI嵌入创建文本块的向量表示。
2. 从这些嵌入创建FAISS向量存储以进行高效的相似性搜索。

### BM25索引创建

1. 从用于向量存储的相同文本块创建BM25索引。
2. 这允许在基于向量的方法之外进行基于关键词的检索。

### 融合检索函数

`fusion_retrieval`函数是此实现的核心：

1. 它接受查询并执行基于向量和基于BM25的检索。
2. 将两种方法的分数标准化到共同的尺度。
3. 计算这些分数的加权组合（由`alpha`参数控制）。
4. 根据组合分数对文档进行排名，并返回前k个结果。

## 此方法的优点

1. 提高检索质量：通过结合语义和基于关键词的搜索，系统可以捕获概念相似性和精确关键词匹配。
2. 灵活性：`alpha`参数允许根据特定用例或查询类型调整向量和关键词搜索之间的平衡。
3. 鲁棒性：组合方法可以有效处理更广泛的查询，减轻单个方法的弱点。
4. 可定制性：系统可以轻松适应使用不同的向量存储或基于关键词的检索方法。

## 结论

融合检索代表了一种强大的文档搜索方法，结合了语义理解和关键词匹配的优势。通过利用基于向量和BM25检索方法，它为信息检索任务提供了更全面和灵活的解决方案。这种方法在概念相似性和关键词相关性都很重要的各个领域都有潜在应用，如学术研究、法律文档搜索或通用搜索引擎。

<div style="text-align: center;">

<img src="../images/fusion_retrieval.svg" alt="Fusion Retrieval" style="width:100%; height:auto;">
</div>

# 包安装和导入

下面的单元格安装运行此notebook所需的所有必要包。


In [ ]:
# 安装所需包
!pip install langchain numpy python-dotenv rank-bm25

In [ ]:
# 克隆仓库以访问辅助函数和评估模块
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# 如果需要使用最新数据运行
# !cp -r RAG_TECHNIQUES/data .

In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain.docstore.document import Document

from typing import List
from rank_bm25 import BM25Okapi
import numpy as np


# 为Colab兼容性替换原始路径追加
from helper_functions import *
from evaluation.evalute_rag import *

# 从.env文件加载环境变量
load_dotenv()

# 设置OpenAI API密钥环境变量
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### 定义文档路径

In [ ]:
# 下载所需数据文件
import os
os.makedirs('data', exist_ok=True)

# 下载此notebook中使用的PDF文档
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [2]:
path = "data/Understanding_Climate_Change.pdf"

### 使用OpenAI嵌入将PDF书籍编码到向量存储中。

    Args:
        path: PDF文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。

    Returns:
        包含编码书籍内容的FAISS向量存储。
    """

    # 加载PDF文档
    loader = PyPDFLoader(path)
    documents = loader.load()

    # 将文档分割成块
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # 创建嵌入和向量存储
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore, cleaned_texts

### 创建向量存储并获取分块文档

In [4]:
vectorstore, cleaned_texts = encode_pdf_and_get_split_documents(path)

### Create a bm25 index for retrieving documents by keywords

In [5]:
def create_bm25_index(documents: List[Document]) -> BM25Okapi:
    """
    从给定文档创建BM25索引。

    BM25（最佳匹配25）是信息检索中使用的排名函数。
    它基于概率检索框架，是对TF-IDF的改进。

    Args:
    documents (List[Document]): 要索引的文档列表。

    Returns:
    BM25Okapi: 可用于BM25评分的索引。
    """
    # 通过空白字符分割对每个文档进行分词
    # 这是一种简单的方法，可以通过更复杂的分词来改进
    tokenized_docs = [doc.page_content.split() for doc in documents]
    return BM25Okapi(tokenized_docs)

In [6]:
bm25 = create_bm25_index(cleaned_texts) # 从清理后的文本（块）创建BM25索引

### Define a function that retrieves both semantically and by keyword, normalizes the scores and gets the top k documents

In [7]:
def fusion_retrieval(vectorstore, bm25, query: str, k: int = 5, alpha: float = 0.5) -> List[Document]:
    """
    执行融合检索，结合基于关键词（BM25）和基于向量的搜索。

    Args:
    vectorstore (VectorStore): 包含文档的向量存储。
    bm25 (BM25Okapi): 预计算的BM25索引。
    query (str): 查询字符串。
    k (int): 要检索的文档数量。
    alpha (float): 向量搜索分数的权重（1-alpha将是BM25分数的权重）。

    Returns:
    List[Document]: 基于组合分数的前k个文档。
    """
    
    epsilon = 1e-8

    # 步骤1：从向量存储获取所有文档
    all_docs = vectorstore.similarity_search("", k=vectorstore.index.ntotal)

    # 步骤2：执行BM25搜索
    bm25_scores = bm25.get_scores(query.split())

    # 步骤3：执行向量搜索
    vector_results = vectorstore.similarity_search_with_score(query, k=len(all_docs))
    
    # 步骤4：标准化分数
    vector_scores = np.array([score for _, score in vector_results])
    vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)

    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) -  np.min(bm25_scores) + epsilon)

    # 步骤5：组合分数
    combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores  

    # 步骤6：对文档进行排名
    sorted_indices = np.argsort(combined_scores)[::-1]
    
    # 步骤7：返回前k个文档
    return [all_docs[i] for i in sorted_indices[:k]]

### 用例示例

In [8]:
# 查询
query = "What are the impacts of climate change on the environment?"

# 执行融合检索
top_docs = fusion_retrieval(vectorstore, bm25, query, k=5, alpha=0.5)
docs_content = [doc.page_content for doc in top_docs]
show_context(docs_content)